## Initialize Libs

In [1]:
#r "E:/_Projects/RateMapSeveritySaber/Ramses.Lights/bin/Release/net8.0/Ramses.Lights.dll"
#r "nuget: Microsoft.Extensions.Logging, 8.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"

Installed Packages Microsoft.Extensions.Logging, 8.0.0 Microsoft.Extensions.Logging.Console, 8.0.0

In [2]:
using MarkovSharp;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using System.IO;
using System.Linq;
using System.Diagnostics;
using System.Text.Json;
using System.Text.Json.Nodes;
using System.Text.Json.Serialization;
using Ramses.Lights;

var loggerFactory = LoggerFactory.Create(builder => builder.AddConsole());
var jIndented = new JsonSerializerOptions { WriteIndented = true };

## Setup Data

In [3]:
var dataset = await MapSelector.ReadMpack<List<List<LightEvent>>>();


In [11]:
// Cleanup data
Parallel.ForEach(dataset, m => {
	m.RemoveAll(e => e.Type is 10 or 14 or 15 || e.Value is < 0 or > ushort.MaxValue);
});
dataset.RemoveAll(m => m.Count == 0);

## Convert to Tokens

In [13]:
public readonly record struct LightToken : IComparable<LightToken>
{
	public static readonly LightToken ZeroLight = default;

	public ushort Value { get; init; }
	public byte Time { get; init; }
	public byte Type { get; init; }
	public bool FloatValue { get; init; }
	public bool Combo { get; init; }

	public static byte CategorizeLightSpeed(float diff) => diff switch
	{
		< 0.050f => 0,
		< 0.100f => 1,
		< 0.200f => 2,
		< 0.500f => 3,
		< 1.000f => 4,
		_ => 5
	};

	public readonly int CompareTo(LightToken other)
	{
		var timeComparison = Time.CompareTo(other.Time);
		if (timeComparison != 0) return timeComparison;
		var valueComparison = Value.CompareTo(other.Value);
		if (valueComparison != 0) return valueComparison;
		var typeComparison = Type.CompareTo(other.Type);
		if (typeComparison != 0) return typeComparison;
		return FloatValue.CompareTo(other.FloatValue);
	}

	public enum LightType : byte
	{
		Strobo,
		Highlight,
		Slow
	}

	public override int GetHashCode() => HashCode.Combine(Value, Time, Type, FloatValue);
}

In [14]:
var tokens = new LightToken[dataset.Count][];

Parallel.ForEach(dataset.Select((m, i) => (m, i)), mi =>
{
	var (m, i) = mi;

	// convert to light tokens
	// take absolute value of time and convert to invervals between each event

	var lightTokens = new LightToken[m.Count];
	var times = m
		.GroupBy(e => float.Round(e.Time, 3))
		.ToDictionary(x => x.Key, x => x.Count());

	for (int j = 0; j < m.Count - 1; j++)
	{
		var currentEvent = m[j];
		var nextEvent = m[j + 1];

		var t = nextEvent.Time - currentEvent.Time;

		lightTokens[j] = new LightToken
		{
			Time = LightToken.CategorizeLightSpeed(t),
			Type = currentEvent.Type,
			Value = checked((ushort)currentEvent.Value),
			FloatValue = currentEvent.HasFloatValue,
			Combo = times[float.Round(currentEvent.Time, 3)] > 1,
		};
	}

	lightTokens[^1] = new LightToken
	{
		Time = 5,
		Value = checked((ushort)m[^1].Value),
		Type = m[^1].Type,
		FloatValue = m[^1].HasFloatValue,
		Combo = times[float.Round(m[^1].Time, 3)] > 1,
	};

	tokens[i] = lightTokens;
});

# Train V2

## Setup Markov Code

In [5]:
using System;
using System.Collections;
using System.Collections.Generic;
using System.Linq;

/// <summary>
/// Represents a state in a Markov chain.
/// </summary>
/// <typeparam name="T">The type of the constituent parts of each state in the Markov chain.</typeparam>
public class ChainState<T> : IEquatable<ChainState<T>>, IList<T>
{
	private readonly T[] items;

	/// <summary>
	/// Initializes a new instance of the <see cref="ChainState{T}"/> class with the specified items.
	/// </summary>
	/// <param name="items">An <see cref="IEnumerable{T}"/> of items to be copied as a single state.</param>
	public ChainState(IEnumerable<T> items)
	{
		if (items == null)
		{
			throw new ArgumentNullException(nameof(items));
		}

		this.items = items.ToArray();
	}

	/// <summary>
	/// Initializes a new instance of the <see cref="ChainState{T}"/> class with the specified items.
	/// </summary>
	/// <param name="items">An array of <typeparamref name="T"/> items to be copied as a single state.</param>
	public ChainState(params T[] items)
	{
		if (items == null)
		{
			throw new ArgumentNullException(nameof(items));
		}

		this.items = new T[items.Length];
		Array.Copy(items, this.items, items.Length);
	}

	/// <inheritdoc />
	public int Count => this.items.Length;

	/// <inheritdoc />
	public bool IsReadOnly => true;

	/// <inheritdoc />
	public T this[int index]
	{
		get { return this.items[index]; }
		set { throw new NotSupportedException(); }
	}

	/// <summary>
	/// Determines whether two specified instances of <see cref="ChainState{T}"/> are not equal.
	/// </summary>
	/// <param name="a">The first <see cref="ChainState{T}"/> to compare.</param>
	/// <param name="b">The second <see cref="ChainState{T}"/> to compare.</param>
	/// <returns>true if <paramref name="a"/> and <paramref name="b"/> do not represent the same state; otherwise, false.</returns>
	public static bool operator !=(ChainState<T> a, ChainState<T> b)
	{
		return !(a == b);
	}

	/// <summary>
	/// Determines whether two specified instances of <see cref="ChainState{T}"/> are equal.
	/// </summary>
	/// <param name="a">The first <see cref="ChainState{T}"/> to compare.</param>
	/// <param name="b">The second <see cref="ChainState{T}"/> to compare.</param>
	/// <returns>true if <paramref name="a"/> and <paramref name="b"/> represent the same state; otherwise, false.</returns>
	public static bool operator ==(ChainState<T> a, ChainState<T> b)
	{
		if (object.ReferenceEquals(a, b))
		{
			return true;
		}
		else if (a is null)
		{
			return false;
		}

		return a.Equals(b);
	}

	/// <inheritdoc />
	public void Add(T item) => throw new NotSupportedException();

	/// <inheritdoc />
	public void Clear() => throw new NotSupportedException();

	/// <inheritdoc />
	public bool Contains(T item) => ((IList<T>)this.items).Contains(item);

	/// <inheritdoc />
	public void CopyTo(T[] array, int arrayIndex) => this.items.CopyTo(array, arrayIndex);

	/// <inheritdoc />
	public override bool Equals(object obj)
	{
		if (obj is ChainState<T> chain)
		{
			return this.Equals(chain);
		}

		return false;
	}

	/// <summary>
	/// Indicates whether the current object is equal to another object of the same type.
	/// </summary>
	/// <param name="other">An object to compare with this object.</param>
	/// <returns>true if the current object is equal to the <paramref name="other"/> parameter; otherwise, false.</returns>
	public bool Equals(ChainState<T> other)
	{
		if (other is null)
		{
			return false;
		}

		if (this.items.Length != other.items.Length)
		{
			return false;
		}

		for (var i = 0; i < this.items.Length; i++)
		{
			if (!this.items[i].Equals(other.items[i]))
			{
				return false;
			}
		}

		return true;
	}

	/// <inheritdoc />
	public IEnumerator<T> GetEnumerator() => ((IList<T>)this.items).GetEnumerator();

	/// <inheritdoc />
	IEnumerator IEnumerable.GetEnumerator() => this.GetEnumerator();

	/// <inheritdoc />
	public override int GetHashCode()
	{
		var code = this.items.Length;

		for (var i = 0; i < this.items.Length; i++)
		{
			code = (code * 37) + this.items[i].GetHashCode();
		}

		return code;
	}

	/// <inheritdoc />
	public int IndexOf(T item) => throw new NotSupportedException();

	/// <inheritdoc />
	public void Insert(int index, T item) => throw new NotSupportedException();

	/// <inheritdoc />
	public bool Remove(T item) => throw new NotSupportedException();

	/// <inheritdoc />
	public void RemoveAt(int index) => throw new NotSupportedException();
}

/// <summary>
/// Builds and walks interconnected states based on a weighted probability.
/// </summary>
/// <typeparam name="T">The type of the constituent parts of each state in the Markov chain.</typeparam>
public class MarkovChain<T>
	where T : IEquatable<T>
{
	private readonly Dictionary<ChainState<T>, Dictionary<T, int>> items = new Dictionary<ChainState<T>, Dictionary<T, int>>();
	private readonly int order;
	private readonly Dictionary<ChainState<T>, int> terminals = new Dictionary<ChainState<T>, int>();

	/// <summary>
	/// Initializes a new instance of the <see cref="MarkovChain{T}"/> class.
	/// </summary>
	/// <param name="order">Indicates the desired order of the <see cref="MarkovChain{T}"/>.</param>
	/// <remarks>
	/// <para>The <paramref name="order"/> of a generator indicates the depth of its internal state.  A generator
	/// with an order of 1 will choose items based on the previous item, a generator with an order of 2
	/// will choose items based on the previous 2 items, and so on.</para>
	/// <para>Zero is not classically a valid order value, but it is allowed here.  Choosing a zero value has the
	/// effect that every state is equivalent to the starting state, and so items will be chosen based on their
	/// total frequency.</para>
	/// </remarks>
	public MarkovChain(int order)
	{
		if (order < 0)
		{
			throw new ArgumentOutOfRangeException(nameof(order));
		}

		this.order = order;
	}

	/// <summary>
	/// Gets the order of the chain.
	/// </summary>
	public int Order => this.order;

	/// <summary>
	/// Adds the items to the generator with a weight of one.
	/// </summary>
	/// <param name="items">The items to add to the generator.</param>
	public void Add(IEnumerable<T> items) => this.Add(items, 1);

	/// <summary>
	/// Adds the items to the generator with the weight specified.
	/// </summary>
	/// <param name="items">The items to add to the generator.</param>
	/// <param name="weight">The weight at which to add the items.</param>
	public void Add(IEnumerable<T> items, int weight)
	{
		if (items == null)
		{
			throw new ArgumentNullException(nameof(items));
		}

		var previous = new Queue<T>();
		foreach (var item in items)
		{
			var key = new ChainState<T>(previous);

			this.Add(key, item, weight);

			previous.Enqueue(item);
			if (previous.Count > this.order)
			{
				previous.Dequeue();
			}
		}

		this.AddTerminalInternal(new ChainState<T>(previous), weight);
	}

	/// <summary>
	/// Adds the item to the generator, with the specified items preceding it.
	/// </summary>
	/// <param name="previous">The items preceding the item.</param>
	/// <param name="item">The item to add.</param>
	/// <remarks>
	/// See <see cref="MarkovChain{T}.Add(IEnumerable{T}, T, int)"/> for remarks.
	/// </remarks>
	public void Add(IEnumerable<T> previous, T item)
	{
		if (previous == null)
		{
			throw new ArgumentNullException(nameof(previous));
		}

		var state = new Queue<T>(previous);
		while (state.Count > this.order)
		{
			state.Dequeue();
		}

		this.Add(new ChainState<T>(state), item, 1);
	}

	/// <summary>
	/// Adds the item to the generator, with the specified state preceding it.
	/// </summary>
	/// <param name="state">The state preceding the item.</param>
	/// <param name="next">The item to add.</param>
	/// <remarks>
	/// See <see cref="MarkovChain{T}.Add(ChainState{T}, T, int)"/> for remarks.
	/// </remarks>
	public void Add(ChainState<T> state, T next) => this.Add(state, next, 1);

	/// <summary>
	/// Adds the item to the generator, with the specified items preceding it and the specified weight.
	/// </summary>
	/// <param name="previous">The items preceding the item.</param>
	/// <param name="item">The item to add.</param>
	/// <param name="weight">The weight of the item to add.</param>
	/// <remarks>
	/// This method does not add all of the preceding states to the generator.
	/// Notably, the empty state is not added, unless the <paramref name="previous"/> parameter is empty.
	/// </remarks>
	public void Add(IEnumerable<T> previous, T item, int weight)
	{
		if (previous == null)
		{
			throw new ArgumentNullException(nameof(previous));
		}

		var state = new Queue<T>(previous);
		while (state.Count > this.order)
		{
			state.Dequeue();
		}

		this.Add(new ChainState<T>(state), item, weight);
	}

	/// <summary>
	/// Adds the item to the generator, with the specified state preceding it and the specified weight.
	/// </summary>
	/// <param name="state">The state preceding the item.</param>
	/// <param name="next">The item to add.</param>
	/// <param name="weight">The weight of the item to add.</param>
	/// <remarks>
	/// This adds the state as-is.  The state may not be reachable if, for example, the
	/// number of items in the state is greater than the order of the generator, or if the
	/// combination of items is not available in the other states of the generator.
	///
	/// A negative weight may be passed, which will have the impact of reducing the weight
	/// of the specified state transition.  This can therefore be used to remove items from
	/// the generator. The resulting weight will never be allowed below zero.
	/// </remarks>
	public virtual void Add(ChainState<T> state, T next, int weight)
	{
		if (state == null)
		{
			throw new ArgumentNullException(nameof(state));
		}

		if (!this.items.TryGetValue(state, out var weights))
		{
			weights = new Dictionary<T, int>();
			this.items.Add(state, weights);
		}

		var newWeight = Math.Max(0, weights.ContainsKey(next)
			? weight + weights[next]
			: weight);
		if (newWeight == 0)
		{
			weights.Remove(next);
			if (weights.Count == 0)
			{
				this.items.Remove(state);
			}
		}
		else
		{
			weights[next] = newWeight;
		}
	}

	/// <summary>
	/// Randomly walks the chain.
	/// </summary>
	/// <returns>An <see cref="IEnumerable{T}"/> of the items chosen.</returns>
	/// <remarks>Assumes an empty starting state.</remarks>
	public IEnumerable<T> Chain() => this.Chain(Enumerable.Empty<T>(), new Random());

	/// <summary>
	/// Randomly walks the chain.
	/// </summary>
	/// <param name="previous">The items preceding the first item in the chain.</param>
	/// <returns>An <see cref="IEnumerable{T}"/> of the items chosen.</returns>
	public IEnumerable<T> Chain(IEnumerable<T> previous) => this.Chain(previous, new Random());

	/// <summary>
	/// Randomly walks the chain.
	/// </summary>
	/// <param name="seed">The seed for the random number generator, used as the random number source for the chain.</param>
	/// <returns>An <see cref="IEnumerable{T}"/> of the items chosen.</returns>
	/// <remarks>Assumes an empty starting state.</remarks>
	public IEnumerable<T> Chain(int seed) => this.Chain(Enumerable.Empty<T>(), new Random(seed));

	/// <summary>
	/// Randomly walks the chain.
	/// </summary>
	/// <param name="previous">The items preceding the first item in the chain.</param>
	/// <param name="seed">The seed for the random number generator, used as the random number source for the chain.</param>
	/// <returns>An <see cref="IEnumerable{T}"/> of the items chosen.</returns>
	public IEnumerable<T> Chain(IEnumerable<T> previous, int seed) => this.Chain(previous, new Random(seed));

	/// <summary>
	/// Randomly walks the chain.
	/// </summary>
	/// <param name="rand">The random number source for the chain.</param>
	/// <returns>An <see cref="IEnumerable{T}"/> of the items chosen.</returns>
	/// <remarks>Assumes an empty starting state.</remarks>
	public IEnumerable<T> Chain(Random rand) => this.Chain(Enumerable.Empty<T>(), rand);

	/// <summary>
	/// Randomly walks the chain.
	/// </summary>
	/// <param name="previous">The items preceding the first item in the chain.</param>
	/// <param name="rand">The random number source for the chain.</param>
	/// <returns>An <see cref="IEnumerable{T}"/> of the items chosen.</returns>
	public IEnumerable<T> Chain(IEnumerable<T> previous, Random rand)
	{
		if (previous == null)
		{
			throw new ArgumentNullException(nameof(previous));
		}
		else if (rand == null)
		{
			throw new ArgumentNullException(nameof(rand));
		}

		var state = new Queue<T>(previous);
		while (true)
		{
			while (state.Count > this.order)
			{
				state.Dequeue();
			}

			var key = new ChainState<T>(state);

			var weights = this.GetNextStatesInternal(key);
			if (weights == null)
			{
				yield break;
			}

			var terminalWeight = this.GetTerminalWeight(key);

			var total = weights.Sum(w => w.Value);
			var value = rand.Next(total + terminalWeight) + 1;

			if (value > total)
			{
				yield break;
			}

			var currentWeight = 0;
			foreach (var nextItem in weights)
			{
				currentWeight += nextItem.Value;
				if (currentWeight >= value)
				{
					yield return nextItem.Key;
					state.Enqueue(nextItem.Key);
					break;
				}
			}
		}
	}

	/// <summary>
	/// Gets the items from the generator that follow from an empty state.
	/// </summary>
	/// <returns>A dictionary of the items and their weight.</returns>
	public Dictionary<T, int> GetInitialStates() => this.GetNextStates(new ChainState<T>(Enumerable.Empty<T>()));

	/// <summary>
	/// Gets the items from the generator that follow from the specified items preceding it.
	/// </summary>
	/// <param name="previous">The items preceding the items of interest.</param>
	/// <returns>A dictionary of the items and their weight.</returns>
	public Dictionary<T, int> GetNextStates(IEnumerable<T> previous)
	{
		var state = new Queue<T>(previous);
		while (state.Count > this.order)
		{
			state.Dequeue();
		}

		return this.GetNextStates(new ChainState<T>(state));
	}

	/// <summary>
	/// Gets the items from the generator that follow from the specified state preceding it.
	/// </summary>
	/// <param name="state">The state preceding the items of interest.</param>
	/// <returns>A dictionary of the items and their weight.</returns>
	public Dictionary<T, int> GetNextStates(ChainState<T> state)
	{
		var weights = this.GetNextStatesInternal(state);
		return weights != null ? new Dictionary<T, int>(weights) : null;
	}

	/// <summary>
	/// Gets all of the states that exist in the generator.
	/// </summary>
	/// <returns>An enumerable collection of <see cref="ChainState{T}"/> containing all of the states in the generator.</returns>
	public virtual IEnumerable<ChainState<T>> GetStates()
	{
		foreach (var state in this.items.Keys)
		{
			yield return state;
		}

		foreach (var state in this.terminals.Keys)
		{
			if (!this.items.ContainsKey(state))
			{
				yield return state;
			}
		}
	}

	/// <summary>
	/// Gets the weight of termination following from the specified items.
	/// </summary>
	/// <param name="previous">The items preceding the items of interest.</param>
	/// <returns>A dictionary of the items and their weight.</returns>
	public int GetTerminalWeight(IEnumerable<T> previous)
	{
		var state = new Queue<T>(previous);
		while (state.Count > this.order)
		{
			state.Dequeue();
		}

		return this.GetTerminalWeight(new ChainState<T>(state));
	}

	/// <summary>
	/// Gets the weights of termination following from the specified state.
	/// </summary>
	/// <param name="state">The state preceding the items of interest.</param>
	/// <returns>A dictionary of the items and their weight.</returns>
	public virtual int GetTerminalWeight(ChainState<T> state)
	{
		this.terminals.TryGetValue(state, out var weight);
		return weight;
	}

	/// <summary>
	/// Add a terminal with the given weight. Can be overridden by inheriting classes.
	/// </summary>
	/// <param name="state">The state preceding the items of interest.</param>
	/// <param name="weight">The weight of the terminal to add.</param>
	protected internal virtual void AddTerminalInternal(ChainState<T> state, int weight)
	{
		var newWeight = Math.Max(0, this.terminals.ContainsKey(state)
			? weight + this.terminals[state]
			: weight);
		if (newWeight == 0)
		{
			this.terminals.Remove(state);
		}
		else
		{
			this.terminals[state] = newWeight;
		}
	}

	/// <summary>
	/// Gets the items from the generator that follow from the specified state preceding it without copying the values.
	/// </summary>
	/// <param name="state">The state preceding the items of interest.</param>
	/// <returns>The raw dictionary of the items and their weight.</returns>
	protected internal virtual Dictionary<T, int> GetNextStatesInternal(ChainState<T> state) =>
		this.items.TryGetValue(state, out var weights)
			? weights
			: null;
}



## Code with Markov 2

In [15]:
var mark = new MarkovChain<LightToken>(2);

foreach (var token in tokens)
{
	mark.Add(token, 1);
}

In [16]:
static T PickRandom<T>(IList<T> list) => list[Random.Shared.Next(list.Count)];

var t1 = PickRandom(PickRandom(tokens));
var t2 = PickRandom(PickRandom(tokens));

LightToken[] dummySeed = [
	new LightToken { Time = 0, Type = t1.Type, Value = t1.Value, FloatValue = false },
	new LightToken { Time = 0, Type = t2.Type, Value = t2.Value, FloatValue = false }
];

List<LightToken> examples = [];

//return mark.Chain(dummySeed).FirstOrDefault();

for (int i = 0; i < 10; i++)
{
	examples.Add(mark.Chain(dummySeed).FirstOrDefault());
}

//Console.WriteLine(JsonSerializer.Serialize(examples, jIndented));

In [20]:
var mapFileFs = File.ReadAllBytes(@"F:\SteamLibrary\steamapps\common\Beat Saber\Beat Saber_Data\CustomWIPLevels\Laur-FairyinStrasbourg\HardStandard.dat");
var mapJson = JsonSerializer.Deserialize<JsonNode>(mapFileFs, jIndented);

var bpm = 138f;
var bps = bpm / 60f;
var spb = 1f / bps;

var noteTimes = mapJson["colorNotes"]
	.AsArray()
	.Select(n => n["b"].GetValue<float>() * spb)
	.ToArray();
var sliderBitsTimes = mapJson["burstSliders"]
	.AsArray()
	.SelectMany(n => {
		var start = n["b"].AsValue().GetValue<float>() * spb;
		var end = n["tb"].AsValue().GetValue<float>() * spb;
		var duration = end - start;
		var bitTime = duration / n["sc"].GetValue<int>();
		return Enumerable.Range(0, n["sc"].GetValue<int>()).Select(i => start + i * bitTime);
	})
	.ToArray();

var allTimes = noteTimes
	.Concat(sliderBitsTimes)
	.Select(n => float.Round(n, 3))
	.Distinct()
	.OrderBy(t => t)
	.ToArray();

record GenElem(float Time, LightToken Token);

List<GenElem> genList = [];
HashSet<LightToken> combo = new();

for (int i = 0; i < allTimes.Length - 1; i++)
{
	combo.Clear();
	for(int j = 0; j < 16; j++)
	{
		var next = mark.Chain(genList.Take(^2..).Select(x => x.Token)).FirstOrDefault();
		if(!combo.Add(next))
		{
			continue;
		}
		if (next == null || next.Combo)
		{
			break;
		}
	}
	
	foreach (var next in combo)
	{
		var t = LightToken.CategorizeLightSpeed(allTimes[i + 1] - allTimes[i]);
		genList.Add(new (allTimes[i], new LightToken
		{
			Time = t,
			Type = next.Type,
			Value = next.Value,
			FloatValue = next.FloatValue
		}));
	}
}

mapJson["basicBeatmapEvents"] = new JsonArray(genList.Select(t => {
	var j = new JsonObject();
	j["b"] = float.Round(t.Time * bps, 3);
	j["et"] = t.Token.Type;
	j["i"] = t.Token.Value;
	j["f"] = 1f;
	return j;
}).ToArray());

var newMap = JsonSerializer.Serialize(mapJson, jIndented);
File.WriteAllText(@"F:\SteamLibrary\steamapps\common\Beat Saber\Beat Saber_Data\CustomWIPLevels\Laur-FairyinStrasbourg\NormalStandard.dat", newMap);

# Train V1

## Test Stuff

In [18]:
using System.Runtime.InteropServices;

// Value -> occurences
var agg = new Dictionary<int, ulong>();

void RunHelp() {
	foreach (var token in tokens.SelectMany(t => t))
	{
		ref var cnt = ref CollectionsMarshal.GetValueRefOrAddDefault(agg, token.Value, out _);
		cnt++;
	}
}
RunHelp();

In [19]:
var strb = new StringBuilder();
foreach (var token in agg.OrderByDescending(x => x.Key))
{
	strb.AppendFormat("{0}: {1}", token.Key, token.Value).AppendLine();
}
strb.ToString()

54764: 2
42069: 30
40995: 14
40401: 1
39339: 8
36573: 8
33333: 5
32791: 12
32767: 2
31969: 6
31250: 10
28404: 6
27431: 8
27430: 8
20001: 992
20000: 2
18289: 8
18287: 8
15876: 2
15751: 4
15000: 4
12544: 21
12313: 2
11520: 116
11013: 11
10201: 1
10000: 4356
9999: 91
9222: 11
9147: 8
9144: 8
9099: 153
9098: 13
9081: 14
9066: 14
9062: 14
8983: 13
8853: 14
8850: 14
8667: 13
8434: 14
8364: 14
8255: 14
8151: 13
8135: 14
7890: 2
7818: 13
7613: 14
7435: 12
7314: 12
7258: 14
7002: 13
6969: 18
6666: 10
6601: 14
6518: 13
6432: 14
6430: 14
6404: 11
6250: 10
6007: 1
6000: 2
5985: 14
5655: 13
5602: 13
5401: 14
5328: 14
5123: 11
5000: 277
4928: 14
4832: 14
4767: 14
4557: 47
4444: 10
4439: 6
4251: 14
4161: 24
4118: 13
4083: 12
4033: 77
4000: 1
3969: 92
3842: 11
3792: 14
3675: 8
3625: 14
3541: 15
3462: 14
3350: 14
3193: 220
3048: 13
3035: 2
2863: 74
2862: 12
2800: 10
2758: 16
2566: 13
2521: 14
2353: 2
2345: 2
2320: 14
2112: 39
2044: 14
2020: 4
2000: 1241
1996: 14
1900: 2
1898: 4
1841: 8
1840: 8
1834: 13